In [ ]:
import keras
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import numpy as np

In [ ]:
# the data, shuffled and split between train and test sets
def load_data(dataset):
    # Load
    (x_train, y_train), (x_test, y_test) = dataset.load_data()
    # Reshape X
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    n_channels = x_train.shape[3] if len(x_train.shape) == 4 else 1
    if K.image_data_format() == 'channels_first':
        s = [n_channels, img_rows, img_cols]
    else:
        s = [img_rows, img_cols, n_channels]
    x_train = x_train.reshape(x_train.shape[0], *s)
    x_test = x_test.reshape(x_test.shape[0], *s)
    # Rescale X
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    # Reshape y
    num_classes = len(np.unique(y_train))
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_train, y_train), (x_test, y_test), img_rows, img_cols

In [ ]:
(x_train, y_train), (x_test, y_test), img_rows, img_cols = load_data(cifar10)

In [ ]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
linear = Sequential()
linear.add(Flatten(input_shape=x_train.shape[1:]))
linear.add(Dense(units=10, activation='softmax'))

In [ ]:
small_cnn = Sequential()

small_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=x_train.shape[1:]))
small_cnn.add(Conv2D(32, (3, 3), activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(64, (3, 3), activation='relu'))
small_cnn.add(Conv2D(64, (3, 3), activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Flatten())

small_cnn.add(Dense(units=10, activation='softmax'))

In [ ]:
#model = linear
model = small_cnn

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
n_train    = 6000
batch_size = 128
epochs     = 10

model.fit(x_train[:n_train], y_train[:n_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
_, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', acc)